# **핵심만 요약한 통계와 머신러닝 파이썬 코드북 개정1판**
- ⓒ2023 AlgoBoni all rights reserved.
- 본 컨텐츠의 저작권은 알고보니에 있습니다. 저작권법에 의해 보호를 받는 저작물이므로 무단 전재와 무단 복제를 금합니다.
- 본 컨텐츠의 종이책은 [교보문고](https://product.kyobobook.co.kr/detail/S000209591909), [예스24](https://www.yes24.com/Product/Goods/122661688), [알라딘](https://www.aladin.co.kr/shop/wproduct.aspx?ISBN=K262935029&start=pnaver_02)에서 구매할 수 있습니다. 종이책에서는 아래 개념 및 코드에 대한 설명과 연습문제를 제공합니다.

# 3. 표본추출, 데이터 분할, 교차검증
## 3-1. 표본추출
### - 단순랜덤추출법

In [390]:
# Scikit learn의 내장 데이터 iris 불러오기
from sklearn.datasets import load_iris
import numpy as np
from pandas import DataFrame

data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = DataFrame(np.c_[data['data'], data['target']], columns = [col.replace(" (cm)", "") for col in iris_cols])
print(iris.head(5))

   sepal length  sepal width  petal length  petal width  target
0           5.1          3.5           1.4          0.2     0.0
1           4.9          3.0           1.4          0.2     0.0
2           4.7          3.2           1.3          0.2     0.0
3           4.6          3.1           1.5          0.2     0.0
4           5.0          3.6           1.4          0.2     0.0


In [391]:
# 복원하지 않고 3개를 랜덤추출
print(iris.sample(n=3, replace = False))

    sepal length  sepal width  petal length  petal width  target
22           4.6          3.6           1.0          0.2     0.0
39           5.1          3.4           1.5          0.2     0.0
47           4.6          3.2           1.4          0.2     0.0


In [392]:
# frac 파라미터로 전체 데이터의 3%를 랜덤추출
print(iris.sample(frac=0.03))

    sepal length  sepal width  petal length  petal width  target
88           5.6          3.0           4.1          1.3     1.0
67           5.8          2.7           4.1          1.0     1.0
2            4.7          3.2           1.3          0.2     0.0
91           6.1          3.0           4.6          1.4     1.0


In [393]:
# 3개의 열(axis=1)을 랜덤 추출
print(iris.sample(3, axis=1).head(3))

   petal length  sepal length  sepal width
0           1.4           5.1          3.5
1           1.4           4.9          3.0
2           1.3           4.7          3.2


In [394]:
# 리스트에서 단순랜덤추출하기
import random
import numpy as np
data_list = [1,2,3,4,5,'a', 'b', 'c']
print("random.sample: ", random.sample(data_list, 4))
print("np.random.choice", np.random.choice(data_list, 4, replace = True))

random.sample:  ['c', 'a', 'b', 3]
np.random.choice ['a' '5' '2' '3']


In [395]:
# [참고] 난수 생성하기
print("0~10 사이의 정수 중 3개의 난수 생성: ", np.random.randint(0, 10, 3))
print("0~1 사이의 실수를 2*2 배열로 생성:\n", np.random.rand(2,2))

0~10 사이의 정수 중 3개의 난수 생성:  [5 6 3]
0~1 사이의 실수를 2*2 배열로 생성:
 [[0.25363176 0.9198194 ]
 [0.34766351 0.73365704]]


### - 계통추출법

In [397]:
# 계통추출법으로 표본 추출
data, n = iris, 8 #모집단 데이터프레임, 추출할 샘플 수
N = len(data) #모집단 데이터 크기
K = N//n #구간 내 샘플 수
index = data[:K].sample(1).index #첫 구간에서 임의로 선택한 샘플 인덱스
#첫 샘플로부터 K개씩 띄어서 각 구간에서 하나씩 샘플을 추출
sys_df = DataFrame()
while len(sys_df) < n:
    sys_df = sys_df.append(data.loc[index, :])
    index += K

print(f"N: {N}")
print(f"n: {n}")
print(f"K: {K}")
print(sys_df)

N: 150
n: 8
K: 18
     sepal length  sepal width  petal length  petal width  target
12            4.8          3.0           1.4          0.1     0.0
30            4.8          3.1           1.6          0.2     0.0
48            5.3          3.7           1.5          0.2     0.0
66            5.6          3.0           4.5          1.5     1.0
84            5.4          3.0           4.5          1.5     1.0
102           7.1          3.0           5.9          2.1     2.0
120           6.9          3.2           5.7          2.3     2.0
138           6.0          3.0           4.8          1.8     2.0


### - 집락추출법

### - 층화추출법

In [404]:
# target을 층 혹은 집락이라고 가정!
# 원본 데이터의 분포 확인
from pandas import DataFrame, concat
print(iris['target'].value_counts())

0.0    50
2.0    50
1.0    50
Name: target, dtype: int64


In [398]:
# 데이터, 층/집락 정보를 가진 컬럼명, 추출표본 개수
data, stratum, sampling_no = iris, 'target', 9

# 비례층화추출법: 원본 데이터의 비율대로 추출
levels = data[stratum].unique()
total = data[stratum].value_counts().sum()
prop_val = data[stratum].value_counts()/total
no = prop_val * sampling_no
result = DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(no[level]))
    result = concat([result, temp_df])
print(result)

     sepal length  sepal width  petal length  petal width  target
17            5.1          3.5           1.4          0.3     0.0
43            5.0          3.5           1.6          0.6     0.0
7             5.0          3.4           1.5          0.2     0.0
82            5.8          2.7           3.9          1.2     1.0
88            5.6          3.0           4.1          1.3     1.0
66            5.6          3.0           4.5          1.5     1.0
101           5.8          2.7           5.1          1.9     2.0
140           6.7          3.1           5.6          2.4     2.0
137           6.4          3.1           5.5          1.8     2.0


In [402]:
# 불비례층화추출법: 임의로 정한 특정 비율대로 샘플링
# 데이터, 층/집락 정보를 가진 컬럼명, 추출표본 개수, 각 층/집락의 비율
data, stratum, sampling_no, proportion = iris, 'target', 10, {0:0.2, 1:0.5, 2:0.3}

levels = list(proportion.keys())
prop_val = np.array(list(proportion.values()))
total = sum(prop_val)
no = prop_val * sampling_no
result = DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(no[level]))
    result = concat([result, temp_df])
print(result)

     sepal length  sepal width  petal length  petal width  target
31            5.4          3.4           1.5          0.4     0.0
21            5.1          3.7           1.5          0.4     0.0
82            5.8          2.7           3.9          1.2     1.0
83            6.0          2.7           5.1          1.6     1.0
64            5.6          2.9           3.6          1.3     1.0
68            6.2          2.2           4.5          1.5     1.0
69            5.6          2.5           3.9          1.1     1.0
111           6.4          2.7           5.3          1.9     2.0
127           6.1          3.0           4.9          1.8     2.0
135           7.7          3.0           6.1          2.3     2.0


## 3-2. 데이터 분할
### - 일반적 데이터 분할 및 홀드아웃 방법

In [ ]:
from sklearn.model_selection import train_test_split
X = iris.drop('target', axis=1)
y = iris.filter(['target'])

# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("X_train(y_train): %d(%d), X_test(y_test): %d(%d)"%(len(X_train), len(y_train), len(X_test), len(y_test)))
print("X_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)), "\n")

# 홀드아웃방법
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
print("X_train(y_train): %d(%d), X_test(y_test): %d(%d)"%(len(X_train), len(y_train), len(X_test), len(y_test)))
print("X_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))

X_train(y_train): 105(105), X_test(y_test): 45(45)
X_train의 비율: 0.70, X_test의 비율: 0.30 

X_train(y_train): 75(75), X_test(y_test): 75(75)
X_train의 비율: 0.50, X_test의 비율: 0.50


### - Shuffle Split

In [ ]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=4)
for i, (train_index, test_index) in enumerate(ss.split(X)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ty_train의 타겟 구성:", Counter(y_train['target']))
    print("\ty_test의 타겟 구성:", Counter(y_test['target']), "\n")

Sample 0 ==> train_index: [ 91 107  42], test_index: [103 109  98]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({0.0: 28, 1.0: 27, 2.0: 20})
	y_test의 타겟 구성: Counter({2.0: 30, 1.0: 23, 0.0: 22}) 

Sample 1 ==> train_index: [130 111 115], test_index: [109   4  43]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({2.0: 29, 1.0: 27, 0.0: 19})
	y_test의 타겟 구성: Counter({0.0: 31, 1.0: 23, 2.0: 21}) 

Sample 2 ==> train_index: [44 53 69], test_index: [ 40  45 134]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({0.0: 28, 1.0: 26, 2.0: 21})
	y_test의 타겟 구성: Counter({2.0: 29, 1.0: 24, 0.0: 22}) 

Sample 3 ==> train_index: [  9 137  54], test_index: [140 130  84]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({2.0: 27, 0.0: 24, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 0.0: 26, 2.0: 23}) 



### - K-fold 분할

In [ ]:
from sklearn.model_selection import KFold
from collections import Counter
#n_splits=fold 개수, shuffle=데이터 분할 전 shuffle 여부
kf = KFold(n_splits=4, shuffle=False) 
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ty_train의 타겟 구성:", Counter(y_train['target']))
    print("\ty_test의 타겟 구성:", Counter(y_test['target']), "\n")

Sample 0 ==> train_index: [38 39 40], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test의 타겟 구성: Counter({0.0: 38}) 

Sample 1 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 0.0: 12}) 

Sample 2 ==> train_index: [0 1 2], test_index: [76 77 78]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test의 타겟 구성: Counter({1.0: 24, 2.0: 13}) 

Sample 3 ==> train_index: [0 1 2], test_index: [113 114 115]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test의 타겟 구성: Counter({2.0: 37}) 



### - Stratified K-fold 분할

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4)
# 분할 시 y를 고려해야 하기 때문에 split에 y를 입력!
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ty_train의 타겟 구성:", Counter(y_train['target']))
    print("\ty_test의 타겟 구성:", Counter(y_test['target']), "\n")

Sample 0 ==> train_index: [13 14 15], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12}) 

Sample 1 ==> train_index: [0 1 2], test_index: [13 14 15]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12}) 

Sample 2 ==> train_index: [0 1 2], test_index: [26 27 28]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12}) 

Sample 3 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12}) 



### -  Group K-fold 분할

In [ ]:
# group 변수가 있는 데이터를 생성
## group의 수준은 g0, g1, g2, g3의 4종류가 있다.
iris2 = iris.copy()
iris2['group'] = iris2['target'].apply(lambda x: f"g{int(np.random.randint(0,4,1))}")
print(Counter(iris2['group']))
print(iris2.head(3))

Counter({'g1': 49, 'g2': 39, 'g3': 32, 'g0': 30})
   sepal length  sepal width  petal length  petal width  target group
0           5.1          3.5           1.4          0.2     0.0    g0
1           4.9          3.0           1.4          0.2     0.0    g3
2           4.7          3.2           1.3          0.2     0.0    g2


In [ ]:
from sklearn.model_selection import GroupKFold
X = iris2.drop(['target', 'group'], axis=1)
y = iris2.filter(['target'])
group = iris2.filter(['group'])
gkf = GroupKFold(n_splits=4)
# 분할 시 group을 고려해야 하기 때문에 split에 group를 입력!
for i, (train_index, test_index) in enumerate(gkf.split(X, y, group)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ttrain의 타겟 구성:", Counter(y_train['target']))
    print("\ttest의 타겟 구성:", Counter(y_test['target']))
    print("\ttrain의 그룹 구성:", Counter(group.iloc[train_index]['group']))
    print("\ttest의 그룹 구성:", Counter(group.iloc[test_index]['group']), "\n")

Sample 0 ==> train_index: [0 1 2], test_index: [3 4 8]
	X_train의 비율: 0.67, X_test의 비율: 0.33
	train의 타겟 구성: Counter({2.0: 36, 0.0: 34, 1.0: 31})
	test의 타겟 구성: Counter({1.0: 19, 0.0: 16, 2.0: 14})
	train의 그룹 구성: Counter({'g2': 39, 'g3': 32, 'g0': 30})
	test의 그룹 구성: Counter({'g1': 49}) 

Sample 1 ==> train_index: [0 1 3], test_index: [ 2  7 12]
	X_train의 비율: 0.74, X_test의 비율: 0.26
	train의 타겟 구성: Counter({0.0: 40, 1.0: 37, 2.0: 34})
	test의 타겟 구성: Counter({2.0: 16, 1.0: 13, 0.0: 10})
	train의 그룹 구성: Counter({'g1': 49, 'g3': 32, 'g0': 30})
	test의 그룹 구성: Counter({'g2': 39}) 

Sample 2 ==> train_index: [0 2 3], test_index: [1 5 6]
	X_train의 비율: 0.79, X_test의 비율: 0.21
	train의 타겟 구성: Counter({0.0: 41, 1.0: 40, 2.0: 37})
	test의 타겟 구성: Counter({2.0: 13, 1.0: 10, 0.0: 9})
	train의 그룹 구성: Counter({'g1': 49, 'g2': 39, 'g0': 30})
	test의 그룹 구성: Counter({'g3': 32}) 

Sample 3 ==> train_index: [1 2 3], test_index: [ 0 10 16]
	X_train의 비율: 0.80, X_test의 비율: 0.20
	train의 타겟 구성: Counter({2.0: 43, 1.0: 42, 0.0

## 3-3. 교차 검증

### - 분할 샘플들로 교차 검증

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
X = iris.drop('target', axis=1)
y = iris['target']
LOGREG = LogisticRegression(max_iter = 300, C = 0.1) # 학습 모델 정의
SKF = StratifiedKFold(n_splits=4) # 데이터 분할 방법 정의
result = cross_validate(LOGREG, X, y, cv = SKF, return_train_score=True) #교차 검증
print(DataFrame(result))

   fit_time  score_time  test_score  train_score
0  0.024765    0.002182    0.894737     0.955357
1  0.017696    0.001794    0.947368     0.964286
2  0.018460    0.001895    0.945946     0.955752
3  0.020559    0.001763    1.000000     0.938053


### - 파라미터 후보들로 교차 검증

In [ ]:
from sklearn.model_selection import GridSearchCV
LOGREG = LogisticRegression(max_iter = 300) # 학습 모델 정의
param_grid={'C':[0.01, 0.1, 1], 'solver':['lbfgs', 'liblinear']} #파라미터 후보 정의
SKF = StratifiedKFold(n_splits=4) # 데이터 분할 방법 정의
grid = GridSearchCV(LOGREG, param_grid, cv = SKF) #교차 검증
grid.fit(X, y)
print("최상의 교차 검증 점수: {:.2f}".format(grid.best_score_))
print("최적의 매개변수: {}".format(grid.best_params_))
print(DataFrame(grid.cv_results_))

최상의 교차 검증 점수: 0.97
최적의 매개변수: {'C': 1, 'solver': 'lbfgs'}
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.012633      0.001947         0.001974        0.000203    0.01   
1       0.002517      0.000178         0.001723        0.000059    0.01   
2       0.018816      0.001285         0.001765        0.000030     0.1   
3       0.002247      0.000101         0.001536        0.000013     0.1   
4       0.027849      0.003706         0.002001        0.000142       1   
5       0.004570      0.002461         0.002332        0.000648       1   

  param_solver                              params  split0_test_score  \
0        lbfgs      {'C': 0.01, 'solver': 'lbfgs'}           0.815789   
1    liblinear  {'C': 0.01, 'solver': 'liblinear'}           0.684211   
2        lbfgs       {'C': 0.1, 'solver': 'lbfgs'}           0.894737   
3    liblinear   {'C': 0.1, 'solver': 'liblinear'}           0.815789   
4        lbfgs         {'C': 1, 'solver': 'lbfgs'}  

# 연습문제

### - 1번 문제 풀이

In [440]:
from pandas import read_csv
df = read_csv('https://raw.githubusercontent.com/algoboni/pythoncodebook1-1/main/practice1_bank.csv')
print(df.head(3))

   age         job  marital  education default  balance housing loan  \
0   30  unemployed  married    primary      no     1787      no   no   
1   33    services  married  secondary      no     4789     yes  yes   
2   35  management   single   tertiary      no     1350     yes   no   

    contact month   y  
0  cellular   oct  no  
1  cellular   may  no  
2  cellular   apr  no  


In [441]:
# 범주형 변수들을 레이블 인코딩을 통해 전처리 한다.
df2 = df.copy()
for col in [i for i in df.columns if df[i].dtypes==object]:
    DICT = dict(zip(df[col].unique(), [i for i in range(df[col].nunique())]))
    df2[col] = df2[col].map(DICT)
print(df.head(3))

# 마지막 변수의 DICT 확인
print("\nDICT 생성 확인")
print(df[col].unique())
print([i for i in range(df[col].nunique())])
print(DICT)

   age         job  marital  education default  balance housing loan  \
0   30  unemployed  married    primary      no     1787      no   no   
1   33    services  married  secondary      no     4789     yes  yes   
2   35  management   single   tertiary      no     1350     yes   no   

    contact month   y  
0  cellular   oct  no  
1  cellular   may  no  
2  cellular   apr  no  

DICT 생성 확인
['no' 'yes']
[0, 1]
{'no': 0, 'yes': 1}


In [442]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier() # 학습 모델 정의
param_grid={'max_depth':[5, 6, 7, 8]} #파라미터 후보 정의
SKF = StratifiedKFold(n_splits=6) # 데이터 분할 방법 정의
grid = GridSearchCV(rf, param_grid, cv = SKF) #교차 검증

X = df2.drop('y', axis=1)
y = df2['y']
grid.fit(X, y)

print("최상의 교차 검증 점수: {:.2f}".format(grid.best_score_))
print("최적의 매개변수: {}".format(grid.best_params_))
print(DataFrame(grid.cv_results_))
# 매개변수들을 활용한 교차분석 결과, max_depth가 7일 때 가장 높은 성능을 보이는 것으로 확인하였다.

최상의 교차 검증 점수: 0.88
최적의 매개변수: {'max_depth': 7}
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.220673      0.022650         0.015120        0.001630   
1       0.224396      0.006214         0.014856        0.000865   
2       0.241268      0.008658         0.015578        0.000848   
3       0.251823      0.004222         0.016674        0.000590   

  param_max_depth            params  split0_test_score  split1_test_score  \
0               5  {'max_depth': 5}           0.885942           0.884615   
1               6  {'max_depth': 6}           0.887268           0.883289   
2               7  {'max_depth': 7}           0.887268           0.883289   
3               8  {'max_depth': 8}           0.885942           0.884615   

   split2_test_score  split3_test_score  split4_test_score  split5_test_score  \
0           0.883289           0.885790           0.884462           0.884462   
1           0.880637           0.887118           0.885790           0